# 1. Tensor examples

In [1]:
import torch
# Scalar
scalar = torch.tensor(5.0)
print(f"Scalar: {scalar}")
print(f"Scalar value: {scalar.item()}")  # Convert to a Python number
print(f"Scalar shape: {scalar.shape}")

Scalar: 5.0
Scalar value: 5.0
Scalar shape: torch.Size([])


In [ ]:
# 1D Tensor (Array)
array = torch.tensor([1.0, 2.0, 3.0, 4.0])
print(f"1D Tensor (Array): {array}")
print(f"Array shape: {array.shape}")
print(f"Array sum: {torch.sum(array)}")

1D Tensor (Array): tensor([1., 2., 3., 4.])
Array shape: torch.Size([4])
Array sum: 10.0


In [ ]:
# 2D Tensor (Matrix)
matrix = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
print(f"2D Tensor (Matrix): {matrix}")
print(f"Matrix shape: {matrix.shape}")
print(f"Matrix transpose: {matrix.t()}")
print(f"Matrix multiplication: {torch.mm(matrix, matrix)}")

2D Tensor (Matrix): tensor([[1., 2.],
        [3., 4.]])
Matrix shape: torch.Size([2, 2])
Matrix transpose: tensor([[1., 3.],
        [2., 4.]])
Matrix multiplication: tensor([[ 7., 10.],
        [15., 22.]])


In [4]:
# 3D Tensor
tensor_3d = torch.randn(2, 3, 1)  # Random 3x4x5 tensor
print(f"3D Tensor: {tensor_3d}")
print(f"3D Tensor shape: {tensor_3d.shape}")

3D Tensor: tensor([[[ 0.3394],
         [ 1.1467],
         [-0.9086]],

        [[ 1.1842],
         [-0.9695],
         [-0.2327]]])
3D Tensor shape: torch.Size([2, 3, 1])


In [ ]:
# Element-wise addition
array1 = torch.tensor([1.0, 2.0, 3.0, 4.0])
array2 = torch.tensor([0.5, 1.5, 2.5, 3.5])
print(f"Element-wise addition: {array1 + array2}")

# Element-wise multiplication
print(f"Element-wise multiplication: {array1 * array2}")

Element-wise addition: tensor([1.5000, 3.5000, 5.5000, 7.5000])
Element-wise multiplication: tensor([ 0.5000,  3.0000,  7.5000, 14.0000])


In [ ]:
# Sum
print(f"Sum of array1: {torch.sum(array1)}")

# Mean
print(f"Mean of array1: {torch.mean(array1)}")

# Maximum
print(f"Max of array1: {torch.max(array1)}")

Sum of array1: 10.0
Mean of array1: 2.5
Max of array1: 4.0


In [ ]:
# transforming a vector with a matrix
matrix = torch.tensor([[1.0, 2.0, 3.0],
                       [4.0, 5.0, 6.0]])

vector = torch.tensor([1.0, 2.0, 3.0])

# Perform matrix-vector multiplication using torch.matmul
result = torch.matmul(matrix, vector)
print(result)

tensor([14., 32.])


# 2. Data loading examples

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
import seaborn as sns

In [8]:
iris = sns.load_dataset("iris")
# iris is a pandas dataset
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [11]:
# get the numpy of the iris dataset
X = iris.drop(columns = ['species']).values
y = iris['species'].astype('category').cat.codes.values
print(X[:5])
print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]


In [16]:
# convert numpy to tensor
# here label is long type, 64bit int, bc pytorch has optimized for it, CrossEntropyLoss expects long type
# I can use torch.Tensor(X), but that will require pyptorch to do type inference, which can potentially
# cause performance issues. Better to specify the exact type
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)
# wrap the X and y tensor to the full data before loading to dataloader
full_dataset = TensorDataset(X_tensor, y_tensor)
# it looks like we can't print out the TensorDataset directly like this
print(full_dataset)

# do train test split
train_size = int(0.8*len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# now can add the data to the dataloader
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False)
# it looks like we can't print out the dataloader directly like this
print(train_loader)


# Minimal pytorch neural network example
1. the inputs are two dimensional vectors
2. two layers of NN, converts to 5 dimensional vectors
3. uses sigmoid to basically do softmax to get probability of being true

In [7]:
# generate the dummy data
import torch
X = torch.randn(100, 2)
y = torch.randint(0, 2, (100, 1)).float()

## define the model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# nn.Module is the base class for all neural networks in pytorch.
# when defining a new neural network class, inherit from nn.Module
class BinaryClassifier(nn.Module):
    def __init__(self):
        # here in the init I only set up the weights. The relu is
        # done layer in functional form

        # creates containers for parameters, buffers, and submodules
        # initializes internal flags like training mode
        super().__init__()

        # The input layer transforms 2d input vector into 3d
        # this matrix is a 3 by 2 matrix (reverse from (2,3))
        self.layer1 = nn.Linear(2, 3)
        self.layer2 = nn.Linear(3, 3)
        self.layer3 = nn.Linear(3, 1)

    def forward(self, x):
        # here x is the input vector
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.sigmoid(self.layer3(x))
        return x

## model training

In [8]:
# instantiate the model, create the loss func and optimizer
model = BinaryClassifier()
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

# start the training loop
# puts the model in training mode. Not necessary here bc when model
# is initiated it's in training model. But good to keep the line
# bc it's needed when model gets complex
model.train()
num_epochs = 1000
for epoch in range(num_epochs):
    # forward pass
    y_pred = model(X)

    # compute the loss
    loss = loss_func(y_pred, y)

    # back propogation and update weights
    # first need to zero the grad. By default pytorch accumulates
    # gradients which is useful for advanced stuff. In this case need
    # to zero the grad in every loop
    optimizer.zero_grad()
    # this step is very interesting, bc it seems to not have touched the
    # model. Here loss is a pytorch tensor, PyTorch keeps track of operations
    # on tensors that require gradients from forward to backward pass,
    # creating a graph that links model parameters to the loss.
    # This step adds value to model.linear.weight.grad which is used to update
    # weights by the optimizer
    loss.backward()
    # once the grad is updated, optimizer updates the weights
    optimizer.step()

    # print loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}: loss={loss.item():.3f}")



Epoch 100/1000: loss=0.643
Epoch 200/1000: loss=0.608
Epoch 300/1000: loss=0.605
Epoch 400/1000: loss=0.604
Epoch 500/1000: loss=0.604
Epoch 600/1000: loss=0.603
Epoch 700/1000: loss=0.603
Epoch 800/1000: loss=0.603
Epoch 900/1000: loss=0.603
Epoch 1000/1000: loss=0.599


## model inference

In [13]:
# puts the model in eval model. It does not disable grad calc but signals
# the model it's in inference model.
# It disables dropout, and for batch normalization it uses mean and variance
# from train data so the statistics is fixed for inference
model.eval()

# by default pytorch calculates gradients for any operations involving tensors
# this isn't necessary during inference, so temporarily disable grad calc
with torch.no_grad():
    test_X = torch.tensor([[0.5,0.6]])
    pred = model(test_X)
    print(pred)

tensor([[0.6184]])
